In [1]:
import sys
import os

sys.path.append("../Pre_processing")

from Data_Preparation_Library import *
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers.core import Dropout

In [3]:
#selectedBatches=["6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]
maxBatchId = 1
selectedBatches=[str(i) for i in range(maxBatchId)]

extra_folder="128_128"
batch_data_object = []
for i in selectedBatches:
    with open(os.path.join(temp_path,extra_folder,'full_data_object_' + i + '.p'), 'rb') as handle:
        batch_data_object+=pickle.load(handle)

data_train = batch_data_object[0:int(len(batch_data_object)*2/3)]
#data_test  = batch_data_object[int(len(batch_data_object)*2/3):]

In [4]:
extra_folder="128_128"
batch_data_test_object = []
for i in range(8):
    with open(os.path.join(temp_path,extra_folder,'test_data_object_' + str(i) + '.p'), 'rb') as handle:
        batch_data_test_object+=pickle.load(handle)

In [5]:
data_test  = batch_data_test_object

In [6]:
def flux(d):
  # TODO: get flux_i from model
    if d.i_image != None:
        flux_i = sum(sum(d.i_image))#
    else:
        flux_i = 0
    flux_g = sum(sum(d.g_image))
    #return flux_i+flux_g, flux_g-flux_i
    return flux_g

In [7]:
# with open(os.path.join(os.path.join(output_path,"results_128_on_128"),"encoder_results_train_v1__all"),'rb') as handle:
#     encoded_imgs1=pickle.load(handle)
#     encoded_imgs_reshaped_train = [e.reshape(-1,1) for e in encoded_imgs1]

# with open(os.path.join(output_path,"encoder_results_test_v2_" + "_".join(selectedBatches)),'rb') as handle:
#     encoded_imgs2=pickle.load(handle)
#     encoded_imgs_reshaped_test = [e.reshape(-1,1) for e in encoded_imgs2]


In [8]:
model_encoder=load_model(os.path.join(os.path.join(output_path,"results_128_on_128"),"encoder_model_v1__all"))

def get_g_Data_for_Autoencoder(data_object):
    X_list=[(data_object[index].g_image_resized_reshaped)/data_object[index].g_image_resized_reshaped.max() for index in range(len(data_object))]
    return np.asarray(X_list),np.asarray(X_list)

In [9]:
X_train_auto_encoder,y_train_auto_encoder=get_g_Data_for_Autoencoder(batch_data_object)

encoded_imgs1=model_encoder.predict(X_train_auto_encoder)
encoded_imgs_reshaped_train = [e.reshape(-1,1) for e in encoded_imgs1]



In [10]:
X_test_auto_encoder,y_test_auto_encoder=get_g_Data_for_Autoencoder(batch_data_test_object)

encoded_imgs2=model_encoder.predict(X_test_auto_encoder)
encoded_imgs_reshaped_test = [e.reshape(-1,1) for e in encoded_imgs2]



In [11]:
# accessible in the class: size of image, maximum of image, normalization factor of g + fitted value of normalization factor for i
# define a function to get the training and testing dataset

def obtain_train_data():
    train_features = []
    train_labels = []
    for i in range(len(data_train)):
        if data_train[i].logMstar!=-99:
            e = encoded_imgs_reshaped_train[i]
            d = data_train[i]
            #f1,f2 = flux(d)
            f = flux(d)
            size_1=d.g_image.shape[0]
            size_2=d.g_image.shape[1]
            img_max=d.g_image.max()
            train_features.append(np.append(e,np.array([f,size_1,size_2,img_max])))
            train_labels.append(d.logMstar)
    return train_features, train_labels

train_features = np.asarray(obtain_train_data()[0])
train_labels = np.asarray(obtain_train_data()[1])

print(train_features.shape, train_labels.shape)

(659, 4100) (659,)


In [65]:
train_labels

array([ 10.589,   9.802,  10.543,  10.246,  10.861,  10.149,  10.995,
        10.502,   8.914,  11.251,  10.712,  10.657,  10.984,   9.939,
        11.093,   9.334,  10.625,  10.37 ,   9.104,   8.46 ,   8.586,
         9.198,   9.231,  11.069,  10.74 ,  10.661,  10.938,  10.761,
         9.667,  10.745,  10.313,  10.603,   8.919,  11.008,   9.728,
         8.893,  11.195,  10.377,  10.559,  10.934,  11.279,  10.612,
        11.004,  10.696,   9.561,  11.321,  10.302,   9.726,  10.911,
         9.991,  10.481,   9.406,  10.13 ,  11.244,   9.162,  11.085,
        11.088,  10.456,  10.243,   9.928,  10.155,   9.794,   9.779,
        10.261,  10.216,  10.116,  11.086,   9.978,   9.515,  10.421,
        11.54 ,  11.006,   9.555,   9.747,  10.796,  11.605,   9.88 ,
        11.308,   9.245,  11.209,  10.612,  10.405,   9.35 ,  11.002,
        11.104,   8.924,  10.674,  10.509,  11.26 ,  11.65 ,  10.896,
        10.666,  11.056,   9.97 ,  10.285,  10.796,  10.933,  10.645,
        10.396,  10.

In [12]:
def obtain_test_data():
    test_features = []
    test_labels = []
    for j in range(len(data_test)):
        if data_test[j].logMstar!=-99:
            e = encoded_imgs_reshaped_test[i]
            d = data_test[j]

        #f1,f2 = flux(d)
            f = flux(d)
            size_1=d.g_image.shape[0]
            size_2=d.g_image.shape[1]
            img_max=d.g_image.max()
            test_features.append(np.append(e,np.array([f,size_1,size_2,img_max])))
            test_labels.append(d.logMstar)

    return test_features, test_labels

test_features = np.asarray(obtain_test_data()[0])
test_labels = np.asarray(obtain_test_data()[1])

print(test_features.shape, test_labels.shape)

(8447, 4100) (8447,)


In [13]:
# ## transform features
# train_features = StandardScaler().fit_transform(train_features)
# test_features = StandardScaler().transform(test_features)
# #features = MinMaxScaler().fit_transform(features)

X_scaler = StandardScaler()
train_features_scaled = X_scaler.fit_transform(train_features)
test_features_scaled = X_scaler.transform(test_features)



In [51]:
train_features_scaled=train_features/(train_features.max(0).T)
test_features_scaled=test_features/(train_features.max(0).T)

In [70]:
model = Sequential()
model.add(Dense(64, activation='sigmoid', input_dim=train_features.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(optimizer='Adadelta',
              loss='mse',
              metrics=['mse'])

In [71]:
# model.fit(train_features, train_labels, epochs=100, batch_size=32,shuffle=True,
#                 validation_data=(test_features, test_labels))

model.fit(train_features_scaled, train_labels, epochs=50, batch_size=32,shuffle=True)

Epoch 1/50
659/659 [==============================] - 0s - loss: nan - mean_squared_error: nan     
Epoch 2/50
659/659 [==============================] - 0s - loss: nan - mean_squared_error: nan     
Epoch 3/50
659/659 [==============================] - 0s - loss: nan - mean_squared_error: nan     
Epoch 4/50
659/659 [==============================] - 0s - loss: nan - mean_squared_error: nan     
Epoch 5/50
659/659 [==============================] - 0s - loss: nan - mean_squared_error: nan     
Epoch 6/50
659/659 [==============================] - 0s - loss: nan - mean_squared_error: nan     
Epoch 7/50
659/659 [==============================] - 0s - loss: nan - mean_squared_error: nan     
Epoch 8/50
659/659 [==============================] - 0s - loss: nan - mean_squared_error: nan     
Epoch 9/50
659/659 [==============================] - 0s - loss: nan - mean_squared_error: nan     
Epoch 10/50
659/659 [==============================] - 0s - loss: nan - mean_squared_error: nan     

In [72]:
from sklearn.metrics import mean_squared_error

test_predict = []
#test_labels_2=[]
for i in range(100):
    if data_test[i].logMstar!=-99:
        test_predict.append(model.predict(test_features_scaled[i].reshape(1,-1))[0][0])
        #test_labels_2.append(test_labels[i])
print(len(test_predict))
#print(np.sqrt(mean_squared_error(np.asarray(test_predict), np.asarray(test_labels_2))))


100


In [73]:
model.predict(test_features_scaled[i].reshape(1,-1))

array([[ nan]], dtype=float32)

In [74]:
test_predict

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [56]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [57]:
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, train_features, train_labels.reshape(-1,1)).mean()
print("Score with the entire dataset = %.2f" % score)

Score with the entire dataset = 0.19


In [58]:
estimator.fit(train_features, train_labels.reshape(-1,1))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [59]:
rf_predictions=estimator.predict(test_features)

In [60]:
rf_predictions[0:10]

array([ 9.93385,  9.93263,  9.93032,  9.93032,  9.93383,  9.89868,
        9.93385,  9.93385,  9.95345,  9.93032])

In [61]:
from sklearn.svm import SVR

In [62]:
estimator_svr=SVR()
estimator_svr.fit(train_features_scaled, train_labels.reshape(-1,1))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
svr_predictions=estimator_svr.predict(test_features_scaled)

In [ ]:
svr_predictions

In [64]:
train_features_scaled

array([[ 0.        ,  0.11857449,  0.        , ...,  0.10682111,
         0.10682111,  0.01848817],
       [ 0.03205357,  0.11624919,  0.        , ...,  0.20849421,
         0.20849421,  0.01067172],
       [ 0.0832845 ,  0.0904537 ,  0.14672078, ...,  0.17760618,
         0.17760618,  0.30655937],
       ..., 
       [ 0.        ,  0.1094537 ,  0.        , ...,  0.1023166 ,
         0.1023166 ,  0.01492923],
       [ 0.15812874,  0.08871009,  0.0179127 , ...,  0.12290862,
         0.12290862,  0.01555259],
       [ 0.09979537,  0.08998998,  0.21549305, ...,  0.09073359,
         0.09073359,  0.0921183 ]])

In [63]:
train_labels

array([ 10.589,   9.802,  10.543,  10.246,  10.861,  10.149,  10.995,
        10.502,   8.914,  11.251,  10.712,  10.657,  10.984,   9.939,
        11.093,   9.334,  10.625,  10.37 ,   9.104,   8.46 ,   8.586,
         9.198,   9.231,  11.069,  10.74 ,  10.661,  10.938,  10.761,
         9.667,  10.745,  10.313,  10.603,   8.919,  11.008,   9.728,
         8.893,  11.195,  10.377,  10.559,  10.934,  11.279,  10.612,
        11.004,  10.696,   9.561,  11.321,  10.302,   9.726,  10.911,
         9.991,  10.481,   9.406,  10.13 ,  11.244,   9.162,  11.085,
        11.088,  10.456,  10.243,   9.928,  10.155,   9.794,   9.779,
        10.261,  10.216,  10.116,  11.086,   9.978,   9.515,  10.421,
        11.54 ,  11.006,   9.555,   9.747,  10.796,  11.605,   9.88 ,
        11.308,   9.245,  11.209,  10.612,  10.405,   9.35 ,  11.002,
        11.104,   8.924,  10.674,  10.509,  11.26 ,  11.65 ,  10.896,
        10.666,  11.056,   9.97 ,  10.285,  10.796,  10.933,  10.645,
        10.396,  10.